In [3]:
import pandas as pd
import numpy as np
import os
import time
import sys
from pathlib import Path
start_time = time.time()
import click

In [4]:
main_path = os.getcwd().split('\\bootcamp_covid')[0].capitalize() #confirma o diretório de instalação do script

In [5]:
#função de diminuição de dimensionalidade de listas
def flatten_list(_2d_list):
    flat_list = []
    for element in _2d_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [6]:
df = pd.read_csv(r'{}\bootcamp_covid\data\external\covid_data\full_grouped.csv'.format(main_path)) 
#A utilização destes dados se da por conta da origem dos dados do kaggle virem desses dados e já foi verificado que os dados são os mesmos,
#com isso será utilizado os dados gerados na variável "df" para o andamento do trabalho
paises = ['Mexico', 'Argentina', 'Ecuador', 'Chile', 'Spain']

In [7]:
# carrega os dados em um dicionário
query_country = df[df['Country/Region'].isin(paises)]

In [1]:
query_country.to_excel('covid_silver.xlsx')

In [8]:
query_country

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
6,2020-01-22,Argentina,0,0,0,0,0,0,0,Americas
35,2020-01-22,Chile,0,0,0,0,0,0,0,Americas
51,2020-01-22,Ecuador,0,0,0,0,0,0,0,Americas
111,2020-01-22,Mexico,0,0,0,0,0,0,0,Americas
157,2020-01-22,Spain,0,0,0,0,0,0,0,Europe
...,...,...,...,...,...,...,...,...,...,...
34975,2020-07-27,Argentina,167416,3059,72575,91782,4890,120,2057,Americas
35004,2020-07-27,Chile,347923,9187,319954,18782,2133,75,1859,Americas
35020,2020-07-27,Ecuador,81161,5532,34896,40733,467,17,0,Americas
35080,2020-07-27,Mexico,395489,44022,303810,47657,4973,342,8588,Americas


In [9]:
#Montagem dos dados
df0 = pd.read_csv(r'{}\bootcamp_covid\data\bronze\twitter_bronze\AR_raw_data.csv'.format(main_path))
df0['Country'] = 'Argentina'
df1 = pd.read_csv(r'{}\bootcamp_covid\data\bronze\twitter_bronze\CH_raw_data.csv'.format(main_path))
df1['Country'] = 'Chile'
df2 = pd.read_csv(r'{}\bootcamp_covid\data\bronze\twitter_bronze\ME_raw_data.csv'.format(main_path))
df2['Country'] = 'Mexico'
df3 = pd.read_csv(r'{}\bootcamp_covid\data\bronze\twitter_bronze\EC_raw_data.csv'.format(main_path))
df3['Country'] = 'Ecuador'
df4 = pd.read_csv(r'{}\bootcamp_covid\data\bronze\twitter_bronze\SP_raw_data.csv'.format(main_path))
df4['Country'] = 'Spain'

In [74]:
all = pd.concat([df0,df1,df2,df3,df4])
all.drop('Unnamed: 0', axis = 1)

,text,score,retweet,like,date,lat,long,lang,text_simm,Country,country
0,La OMS estima que no habrá vacuna contra el co...,0.0,0,1,2020-02-11,-34.604450,-58.446244,es,0.999993,Argentina,NaN
1,"Coronavirus: covid-19, la enfermedad del nuevo...",0.0,0,0,2020-02-11,-34.604450,-58.446244,es,0.999995,Argentina,NaN
2,"No digas virus chino o enfermedad china, porqu...",0.0,0,0,2020-02-11,-34.604450,-58.446244,es,0.999997,Argentina,NaN
3,Coronavirus Covid-19: paciente contagiada fue ...,0.0,0,2,2020-02-12,-32.951572,-60.693506,es,0.999996,Argentina,NaN
4,Coronavirus Covid-19: paciente contagiada fue ...,0.0,1,1,2020-02-12,-32.951572,-60.693506,es,0.999996,Argentina,NaN
...,...,...,...,...,...,...,...,...,...,...,...
66171,@Cesar_Vargass @RogerAderly Igualmente aunque ...,0.0,0,1,12/31/2021,NaN,NaN,es,0.999996,Spain,Spain
66172,"Covid, covid , tengo covid y di positivo el dí...",0.0,0,0,12/31/2021,NaN,NaN,es,0.999997,Spain,Spain
66173,"@manupapay Tengo las 3 vacunas, estuve con 4 p...",0.0,0,1,12/31/2021,NaN,NaN,es,0.999997,Spain,Spain
66174,Espero que el 2022 sea un año mejor que este 2...,0.0,0,6,12/31/2021,NaN,NaN,es,0.999997,Spain,Spain


In [170]:
all.to_excel('twitter_data.xlsx')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))